In [1]:
cd /home/olga/pureScratch/olgabot-maca/facs/

/mnt/pureScratch/olga/olgabot-maca/facs


In [2]:
ls -lha

total 122G
drwxrwxr-x 1 olga olga    0 Aug 12 06:29 ./
drwxrwxr-x 1 olga olga    0 Aug  1 01:58 ../
drwxrwxr-x 1 olga olga    0 Aug 12 06:29 .sbt.tabula-muris-k21/
drwxrwxr-x 1 olga olga    0 Aug  1 16:45 sourmash/
-rw-rw-r-- 1 olga olga  47G Aug  6 19:18 sourmash_compare_k21.csv
-rw-rw-r-- 1 olga olga  14G Aug  7 11:51 sourmash_compare_k21_ignore_abundance.csv
-rw-rw-r-- 1 olga olga  48G Aug  6 19:21 sourmash_compare_k31.csv
-rw-rw-r-- 1 olga olga 2.5K Aug  9 14:30 sourmash_compare_k31.hdf5
-rw-rw-r-- 1 olga olga  14G Aug  7 11:51 sourmash_compare_k31_ignore_abundance.csv
-rw-rw-r-- 1 olga olga  13M Aug 12 06:50 tabula-muris-k21.sbt.json


In [3]:
# !ls -lha .sbt.tabula-muris-k21/ | head

In [74]:
import glob
import itertools
from itertools import groupby
import math

import numpy as np
import scipy.sparse
from umap.umap_ import smooth_knn_dist, compute_membership_strengths

from umap.spectral import spectral_layout

%load_ext autoreload
%autoreload 2

import sourmash

from sourmash import signature as sig
from sourmash.sbt import Leaf
from sourmash.sbtmh import SigLeaf, create_sbt_index, load_sbt_index
from sourmash import sourmash_args
from sourmash.logging import notify
from sourmash import knn
from sourmash import umap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Settings for this session

In [5]:

# --- Which signatures to use? --- #
ksize = 21
moltype = 'DNA'

# --- How to build sequence bloom tree? --- #
bf_size = 1e5
n_children = 2
scaled = False

# --- How to build K-nearest neighbor graph on similarities? --- #
n_neighbors = 5
ignore_abundance = True
downsample = False

# --- UMAP settings ---
n_epochs = 0

In [96]:
%time tree = load_sbt_index("tabula-muris-k21.sbt.json")
tree

CPU times: user 760 ms, sys: 116 ms, total: 876 ms
Wall time: 876 ms


In [ ]:
len(tree.nodes)

In [97]:
set(type(node) for node in tree.nodes.values())

{sourmash.sbt.Node, sourmash.sbtmh.SigLeaf}

In [99]:
sum(1 for _ in knn.get_leaves(tree))

50312

In [108]:
sum(1 for node in tree.nodes.values() if isinstance(node, (Leaf, SigLeaf)))

50312

In [104]:
# sum(1 for node in tree.nodes.values() if hasattr(node, 'data.minhash'))

In [106]:
%%time
adjacencies = tree.nearest_neighbor_adjacencies(n_neighbors=n_neighbors, 
                                                ignore_abundance=ignore_abundance, 
                                                downsample=downsample)
print(len(adjacencies))

0
CPU times: user 45.5 s, sys: 36 ms, total: 45.6 s
Wall time: 45.7 s


In [107]:
len(adjacencies)

0

In [109]:
sum(1 for node in tree.nodes.values() if isinstance(node, (Leaf, SigLeaf)))

50312

In [95]:
# %%time

# adjacencies = knn.nearest_neighbors(tree, n_neighbors=n_neighbors,
#                                              ignore_abundance=ignore_abundance,
#                                              downsample=downsample)
# print(len(adjacencies))

KeyboardInterrupt: 

In [89]:
len(adjacencies)

0

In [88]:
adjacencies[:10]

[]

In [12]:
u_leaves = set(u for (u, v, similarity) in adjacencies)

In [13]:
'P9-MAA001884-3_38_F-1-1_S151' in u_leaves

True

In [81]:
leaves = list(knn.get_leaves(tree))
len(leaves)

0

In [82]:
sum(1 for _ in knn.get_leaves(tree))

0

In [85]:
len(tree.nodes)

100623

In [93]:
tree.nodes[10]

In [90]:
set(type(node) for node in tree.nodes.values())

{sourmash.sbt.Node, sourmash.sbtmh.SigLeaf}

In [92]:
sum(1 for node in tree.nodes.values() if hasattr(node, 'minhash'))

0

In [33]:
leaf_to_position = dict(
        (node.data.name(), position) for position, node in tree.nodes.items()
        if isinstance(node, Leaf))
print(len(leaf_to_position))
leaf_to_index = dict((name, i) for i, name in enumerate(leaf_to_position.keys()))
len(leaf_to_index)

50312


50312

In [63]:
knn_indices = []
knn_dists = []

for u, items in groupby(adjacencies, key=lambda x: x[0]):
    knn_indices_line = []
    knn_dists_line = []
    for u, v, similarity in items:
        index = leaf_to_index[v]
        if not isinstance(index, int):
            print(f"index {index} is not an integer")
        knn_indices_line.append(index)

        # Dissimilarity = 1-similarity
        dissimilarity = float(1 - similarity)
        if not isinstance(dissimilarity, float):
            print(f"dissimilarity {dissimilarity} is not a float")
        knn_dists_line.append(dissimilarity)
#     knn_indices_line = np.array(knn_indices_line)
#     knn_dists_line = np.array(knn_dists_line)
    knn_indices.append(knn_indices_line)
    knn_dists.append(knn_dists_line)
knn_indices = np.array(knn_indices, dtype='int')
knn_dists = np.array(knn_dists)

ValueError: setting an array element with a sequence.

In [66]:
[x for x in knn_indices if len(x) != n_neighbors]

[[231],
 [652, 650, 649, 651],
 [784, 778, 790, 789],
 [1224, 1230, 1239],
 [1238, 1230],
 [1252, 1250],
 [1368, 1376, 1378],
 [1393],
 [1400, 1404, 1409],
 [1429],
 [1529, 1541],
 [1584, 1586],
 [1784, 1790, 1798],
 [2074],
 [2177],
 [2215],
 [2242, 2239, 2235],
 [2268, 2267, 2275],
 [2312, 2316, 2326, 2323],
 [2536, 2540],
 [2617],
 [2799],
 [3572, 3566, 3563],
 [3776, 3774, 3778, 3781],
 [4004, 3998],
 [4264, 4274, 4273],
 [4365],
 [4432, 4436],
 [4432, 4438, 4425],
 [4860, 4866, 4857, 4869],
 [5260],
 [5373],
 [5572, 5573],
 [5824],
 [5966, 5970, 5973, 5975],
 [6046, 6041],
 [6058, 6059, 6071, 6065],
 [6064, 6059],
 [6139, 6149],
 [6802, 6795, 6805],
 [7368, 7371, 7375, 7377],
 [7584, 7581],
 [7726, 7730, 7729],
 [8512, 8514, 8509],
 [8514, 8505],
 [9206],
 [9232, 9236, 9239, 9237],
 [9381],
 [9472, 9478, 9474, 9475],
 [9766, 9758],
 [9825, 9817],
 [9872, 9870, 9865],
 [10300, 10310, 10298],
 [10466, 10462, 10457],
 [10712, 10716, 10713],
 [10874, 10882, 10873, 10879],
 [10976, 109

In [61]:
knn_indices.dtype

dtype('O')

In [62]:
knn_indices.astype(int)

ValueError: setting an array element with a sequence.

In [60]:
knn_dists.dtype

dtype('O')

In [54]:
knn_dists[0].dtype

dtype('float64')

In [50]:
knn_indices

array([array([50307, 50305, 50297, 50301, 50303]),
       array([50307, 50305, 50297, 50301, 50303]),
       array([50307, 50305, 50297, 50301, 50303]), ...,
       array([50307, 50305, 50297, 50301, 50303]),
       array([50307, 50305, 50297, 50301, 50303]),
       array([50307, 50305, 50297, 50301, 50303])], dtype=object)

In [79]:
knn_dists.astype(float)

AttributeError: 'list' object has no attribute 'astype'

In [44]:
knn_dists[:10]

[[0.96, 0.956, 0.972, 0.966, 0.966],
 [0.958, 0.96, 0.958, 0.956, 0.972],
 [0.952, 0.964, 0.98, 0.966, 0.972],
 [0.972, 0.972, 0.982, 0.96, 0.96],
 [0.972, 0.968, 0.974, 0.964, 0.962],
 [0.968, 0.97, 0.982, 0.978, 0.976],
 [0.964, 0.926, 0.964, 0.966, 0.956],
 [0.958, 0.954, 0.968, 0.974, 0.98],
 [0.944, 0.954, 0.9299999999999999, 0.9299999999999999, 0.924],
 [0.964, 0.964, 0.964, 0.97, 0.964]]

In [35]:
leaf_to_index['P9-MAA001884-3_38_F-1-1_S151']

50307

In [36]:
leaf_to_position['P9-MAA001884-3_38_F-1-1_S151']

100618

In [78]:
knn_indices.dtypes

AttributeError: 'list' object has no attribute 'dtypes'

In [76]:
graph = umap.fuzzy_simplicial_set(knn_indices, knn_dists, n_neighbors)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'shape' of type reflected list(reflected list(float64))

File "../../../../../home/olga/miniconda3/envs/sourmash-sbt2knn/lib/python3.7/site-packages/umap/umap_.py", line 93:
def smooth_knn_dist(distances, k, n_iter=64, local_connectivity=1.0, bandwidth=1.0):
    <source elided>
    target = np.log2(k) * bandwidth
    rho = np.zeros(distances.shape[0])
    ^

[1] During: typing of get attribute at /home/olga/miniconda3/envs/sourmash-sbt2knn/lib/python3.7/site-packages/umap/umap_.py (93)

File "../../../../../home/olga/miniconda3/envs/sourmash-sbt2knn/lib/python3.7/site-packages/umap/umap_.py", line 93:
def smooth_knn_dist(distances, k, n_iter=64, local_connectivity=1.0, bandwidth=1.0):
    <source elided>
    target = np.log2(k) * bandwidth
    rho = np.zeros(distances.shape[0])
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an 

In [40]:
# knn_indices, knn_dists, leaf_to_index = \
#     sourmash_knn.similarity_adjacency_to_knn(adjacencies, tree)
# print(f"knn_indices.shape: {knn_indices.shape}")
# print(f"knn_dists.shape: {knn_dists.shape}")
# print(f"len(leaf_to_index): {len(leaf_to_index)}")

In [21]:
%debug

> /home/olga/code/sourmash/sourmash/knn.py(75)similarity_adjacency_to_knn()
     73         knn_dists_line = []
     74         for u, v, similarity in items:
---> 75             knn_indices_line.append(leaf_to_index[v])
     76 
     77             # Dissimilarity = 1-similarity



ipdb>  dir()


['adjacencies', 'items', 'knn_dists', 'knn_dists_line', 'knn_indices', 'knn_indices_line', 'leaf_to_index', 'similarity', 'tree', 'u', 'v']


ipdb>  len(leaf_to_index)


0


ipdb>  q 


In [ ]:
index_to_leaf = dict(zip(leaf_to_index.values(), leaf_to_index.keys()))

In [ ]:
graph = sourmash_knn.make_search_graph(knn_indices, knn_dists)

In [ ]:
graph, n_epochs = sourmash_knn._clean_graph(graph, n_epochs)



## Get connected components and similarity_adjacency_to_knnem

In [ ]:
def convert_name_to_tree_position(tree, name):
    for position, node in tree.nodes.items():
        if node.data.name() == name:
            return position

label_to_tree_position = dict((name, convert_name_to_tree_position(name)) for name in leaf_to_index.keys())
len(label_to_tree_position)
label_to_tree_position['P9-MAA001884-3_38_F-1-1_S151']

In [15]:
n_connected_components, labels = scipy.sparse.csgraph.connected_components(graph)

for label in range(n_connected_components):
    graph_index_of_label = np.where(component_labels == label)
    tree_position
    
    first_sig = tree.get(graph)
    for i in graph_index_of_label[1:]:
        

SyntaxError: invalid syntax (<ipython-input-15-c5d4e7a2d9ca>, line 4)

In [ ]:
np.where(component_labels == label)